In [ ]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

import sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
import numpy as np
import glob
import tqdm

from torch.utils.tensorboard import SummaryWriter

pd.options.mode.chained_assignment = None


# Feature Engineering

In [ ]:
'''
    TRD_DD : Date
    ISU_CD : Stock Code
    ISU_NM : Stock Name
    TDD_CLSPRC : Closing Price
    TDD_OPNPRC : Opening Price
    TDD_HGPRC : High Price
    TDD_LWPRC : Low Price
    MKTCAP : Market Capitalization
    ACC_TRDVOL : Trading Volume
    EPS : Earnings Per Share
    PER : Price-Earnings Ratio
    BPS : Book Value Per Share
    PBR : Price-Book Ratio
    DPS : Dividends Per Share
    DVD_YLD : Dividend Yield

'''

In [ ]:

def load_and_merge_csv_files(data_directory, preprocessed_directory, file_limit=None):
    data_files = glob.glob(os.path.join(data_directory, "*.csv"))
    preprocessed_files = glob.glob(os.path.join(preprocessed_directory, "*.csv"))
    
    data_files = data_files[:file_limit]
    preprocessed_files = preprocessed_files[:file_limit]
    
    merged_dfs = []
    
    for data_file in data_files:
        file_name = os.path.basename(data_file)
        
        preprocessed_file_name = file_name.replace('.csv', '_preprocessed.csv')
        preprocessed_file_path = os.path.join(preprocessed_directory, preprocessed_file_name)
        
        if preprocessed_file_path in preprocessed_files:
            df_data = pd.read_csv(data_file)
            df_preprocessed = pd.read_csv(preprocessed_file_path)
            
            merged_df = pd.merge(df_data, df_preprocessed, on='TRD_DD', suffixes=('_data', '_preprocessed'))
            merged_dfs.append(merged_df)
    
    total_df = pd.concat(merged_dfs, ignore_index=True)
    
    return total_df

data_directory = path_append + "../data/KR_Data/data"
preprocessed_directory = path_append + "../data/KR_Data/preprocessed"
total_df = load_and_merge_csv_files(data_directory, preprocessed_directory)

total_df.head()


In [ ]:
# Assuming total_df is already defined and merged from previous steps

# Reverse the DataFrame to sort dates from past to present
total_df = total_df[::-1].reset_index(drop=True)

# Split the "TRD_DD" column into year, month, and day columns
total_df[["Y", "M", "D"]] = total_df["TRD_DD"].str.split("/", expand=True)

# Drop the original "TRD_DD" column
total_df = total_df.drop("TRD_DD", axis=1)

# Rearrange columns to have year, month, and day first
total_df = total_df[["Y", "M", "D"] + total_df.columns[:-3].to_list()]


In [ ]:
# Create a new 'Date' column by combining 'Y', 'M', 'D' columns
total_df['Date'] = pd.to_datetime(total_df[['Y', 'M', 'D']].rename(columns={'Y': 'year', 'M': 'month', 'D': 'day'}))

# Set 'Date' as the index
total_df.set_index('Date', inplace=True)

# Create a 'count_day' column that represents the number of days from the first date
total_df['count_day'] = (total_df.index - total_df.index.min()).days

# Drop the 'Y', 'M', 'Day' columns as they're no longer needed
total_df.drop(columns=['Y', 'M', 'D'], inplace=True)

# Reorder the columns to make 'count_day' first
cols = ['count_day'] + [col for col in total_df.columns if col != 'count_day']
total_df = total_df[cols]

total_df.head()

In [ ]:
total_df.drop(['ISU_CD'], axis=1, inplace=True)

In [ ]:
total_df.reset_index(drop=True, inplace=True)

In [ ]:
total_df.info()

In [ ]:
# Display non-NaN values of the columns to be dropped (for verification)
print("EPS non-NaN values:\n", total_df["EPS"].dropna())
print("PER non-NaN values:\n", total_df["PER"].dropna())
print("BPS non-NaN values:\n", total_df["BPS"].dropna())
print("PBR non-NaN values:\n", total_df["PBR"].dropna())
print("DPS non-NaN values:\n", total_df["DPS"].dropna())
print("DVD_YLD non-NaN values:\n", total_df["DVD_YLD"].dropna())

# Drop the unusable columns
total_df = total_df.drop(["EPS", "PER", "BPS", "PBR", "DPS", "DVD_YLD"], axis=1)


In [ ]:
import pandas as pd

# Assuming total_df is already defined and filled with NaN values replaced by 0
# total_df = ...

# 1) Set TREND to 0 for any value that is not -1, 0, or 1
total_df.loc[~total_df["TREND"].isin([-1, 0, 1]), "TREND"] = 0

# 2) Set TREND to -1 for negative values and 1 for positive values
total_df.loc[total_df["TREND"] < 0, "TREND"] = -1
total_df.loc[total_df["TREND"] > 0, "TREND"] = 1

# 3) Adjust TREND values based on the specified conditions
total_df.loc[total_df["TREND"] <= -0.5, "TREND"] = -1
total_df.loc[total_df["TREND"] >= 0.5, "TREND"] = 1
total_df.loc[(total_df["TREND"] > -0.5) & (total_df["TREND"] < 0.5), "TREND"] = 0

# Check the unique values in the TREND column and their counts
unique_trends = set(total_df["TREND"])
trend_counts = total_df["TREND"].value_counts()

# Print the unique values and their counts
print("Unique TREND values:", unique_trends)
print("TREND value counts:\n", trend_counts)



In [ ]:
total_df["TREND"] += 1

In [ ]:
total_df["TREND"] = total_df["TREND"].convert_dtypes(int)
total_df["TREND"]

In [ ]:
import pandas as pd

# List of columns to convert from strings to numeric values
columns_to_convert = ["TDD_CLSPRC", "TDD_OPNPRC", "TDD_HGPRC", "TDD_LWPRC", "MKTCAP", "ACC_TRDVOL"]

# Convert the columns to numeric values
for col in columns_to_convert:
    total_df[col] = total_df[col].str.replace(pat=r'[^0-9]', repl=r'' ,regex=True).apply(pd.to_numeric)


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Assuming total_df is already defined and filled with NaN values replaced by 0
# total_df = ...

# Define the scalers
mm = MinMaxScaler()
sc = RobustScaler()

# total_df = total_df[['ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP', 'ACC_TRDVOL', 'TREND']]

# # Apply MinMax scaling to the specified columns
minmax_cols = ["count_day", "TDD_CLSPRC", "TDD_OPNPRC", "TDD_HGPRC", "TDD_LWPRC"]
for col in minmax_cols:
    total_df[col] = mm.fit_transform(total_df[col].values.reshape(-1, 1))

# Apply Robust scaling to the specified columns
robust_cols = ["MKTCAP", "ACC_TRDVOL"]
for col in robust_cols:
    total_df[col] = sc.fit_transform(total_df[col].values.reshape(-1, 1))


In [ ]:
total_df.head()

In [ ]:
total_df.info()

In [ ]:
# Ensure 'ISU_NM' is of string type
total_df["ISU_NM"] = total_df["ISU_NM"].astype(str)

# Calculate where 'ISU_NM' column changes value
isu_nm_changes = total_df['ISU_NM'].shift() != total_df['ISU_NM']
change_indices = [0] + isu_nm_changes[isu_nm_changes].index.tolist() + [len(total_df)]

# Compute pairs of (start, end) indices
segment_pairs = [(change_indices[i], change_indices[i+1]) for i in range(len(change_indices) - 1)]

print("Pairs of (start, end) indices:", segment_pairs)

In [ ]:
# Drop ISU_NM if it exists
if "ISU_NM" in total_df.columns:
    total_df = total_df.drop("ISU_NM", axis=1)
else:
    print("Column 'ISU_NM' not found in DataFrame")

In [ ]:
import dask.dataframe as dd

# Convert to Dask DataFrame
ddf = dd.from_pandas(total_df, npartitions=10)

# Apply pd.to_numeric in a distributed manner
ddf_numeric = ddf.map_partitions(lambda df: df.apply(pd.to_numeric, errors='coerce')).compute()

ddf_numeric.info()


In [ ]:
import pandas as pd
import torch

def convert_nullable_int_columns(df):

    int_columns = df.select_dtypes(include=['Int64']).columns
    for col in int_columns:
        df[col] = df[col].astype('int64')
    return df

def process_dataframe(df, segments):

    df_numeric = df.apply(pd.to_numeric, errors='coerce')

    df_numeric = convert_nullable_int_columns(df_numeric)

    df_numeric = df_numeric.dropna()

    df_tensor = torch.tensor(df_numeric.values, dtype=torch.float64).cuda()

    processed_tensor = torch.cat([df_tensor[start:end] for start, end in segments], dim=0)

    processed_df = pd.DataFrame(processed_tensor.cpu().numpy(), columns=df_numeric.columns)

    return processed_df, segments

total_df, segment_pairs = process_dataframe(total_df, segment_pairs)


# Data Overview and Usage Guide

In [1]:
import pandas as pd
import pickle


use_save_file = True
if use_save_file:
    import sys
    path_append = "../"
    sys.path.append(path_append)  # Go up one directory from where you are.

    from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelEncoder
    from sklearn.model_selection import train_test_split

    import matplotlib.pyplot as plt
    import pandas as pd

    from torch.utils.tensorboard import SummaryWriter
    
    total_df = pd.read_parquet('processed_total_df.parquet')
    # # Load the list from the pickle file
    with open('segment_pairs.pkl', 'rb') as f:
        segment_pairs = pickle.load(f)
else:
    # Save the processed DataFrame to a Parquet file
    total_df.to_parquet('processed_total_df.parquet')
    # save segment_pairs
    with open('segment_pairs.pkl', 'wb') as f:
        pickle.dump(segment_pairs, f)

c:\Users\CCNets-team\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\CCNets-team\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Data Overview and Usage Guide

"""
- Data Overview
    Preprocessed Data: total_df
    Categorical Columns: Y, M, D, ISU_CD, GDC_sig, RSI_sig, ROC_sig, MAP_sig, STC_sig
    Numerical Columns: TDD_CLSPRC, TDD_OPNPRC, TDD_HGPRC, TDD_LWPRC, MKTCAP, ACC_TRDVOL
    Label: TREND

- Considerations:
    1) It is recommended to use embedding techniques for categorical data.
    2) Labels:
        NaN values have been replaced with 0.

        2-1) Label Processing:
            How to handle -1, 0, 1 depends on the definition.
            ● Classification of -1, 0, 1:
                Commonly, the label being discrete is an issue.
                (1) Set to -1 for values less than 0, and 1 for values greater than 0.
                    # Ratio of -1, 0, 1 = 1397:1440:55
                    : This results in very frequent trading.

                (2) Use only -1, 0, 1.
                    # Ratio of -1, 0, 1 = 76:2740:76
                    : This might cause the model to miss buying opportunities when it should, making it difficult for the model to make accurate predictions.

                (3) Set to -1 for values less than -0.5, and 1 for values greater than 0.5, otherwise 0.
                    # Ratio of -1, 0, 1 = 752:1409:731
                    : (Current preprocessing state) This provides a somewhat balanced ratio.

            ● Regression:
                Keep the label as it is.
                (1) The model performs regression and decides whether to buy or sell based on the predicted increase or decrease.

    3) The utility of GDC, RSI, ROC, MAP, STC indicators for learning is uncertain.
"""


'\n- Data Overview\n    Preprocessed Data: total_df\n    Categorical Columns: Y, M, D, ISU_CD, GDC_sig, RSI_sig, ROC_sig, MAP_sig, STC_sig\n    Numerical Columns: TDD_CLSPRC, TDD_OPNPRC, TDD_HGPRC, TDD_LWPRC, MKTCAP, ACC_TRDVOL\n    Label: TREND\n\n- Considerations:\n    1) It is recommended to use embedding techniques for categorical data.\n    2) Labels:\n        NaN values have been replaced with 0.\n\n        2-1) Label Processing:\n            How to handle -1, 0, 1 depends on the definition.\n            ● Classification of -1, 0, 1:\n                Commonly, the label being discrete is an issue.\n                (1) Set to -1 for values less than 0, and 1 for values greater than 0.\n                    # Ratio of -1, 0, 1 = 1397:1440:55\n                    : This results in very frequent trading.\n\n                (2) Use only -1, 0, 1.\n                    # Ratio of -1, 0, 1 = 76:2740:76\n                    : This might cause the model to miss buying opportunities when it 

In [5]:
import torch
import random
import torch.nn.functional as F
from torch.utils.data import Dataset

class SequentialDataset(Dataset):
    def __init__(self, df, segment_pairs, max_window_size):
        self.df = df
        self.segment_pairs = segment_pairs
        self.max_window_size = max_window_size
        self.min_window_size = max_window_size // 2
        
        # Compute the total number of possible subsequences
        self.subsequence_lengths = [
            max(0, end - start - self.min_window_size + 1)
            for start, end in self.segment_pairs
        ]
        self.cumulative_lengths = [sum(self.subsequence_lengths[:i+1]) for i in range(len(self.subsequence_lengths))]

    def __len__(self):
        return self.cumulative_lengths[-1]

    def __getitem__(self, idx):
        # Find the appropriate segment for the given idx
        segment_idx = next(i for i, cum_len in enumerate(self.cumulative_lengths) if cum_len > idx)
        
        # Adjust idx to be relative to the found segment
        if segment_idx > 0:
            idx -= self.cumulative_lengths[segment_idx - 1]
        
        start_idx, end_idx = self.segment_pairs[segment_idx]
        actual_start_idx = start_idx + idx
        
        # Ensure we stay within the bounds of the current segment
        window_size = random.randint(self.min_window_size, self.max_window_size)
        seq_end_idx = min(actual_start_idx + window_size, end_idx)

        seq = self.df.iloc[actual_start_idx:seq_end_idx]

        X = seq.drop(['TREND'], axis=1)[1:]
        y = seq['TREND'][:-1]
        
        # y = torch.tensor(y.values, dtype=torch.float32)
        X = torch.tensor(X.values, dtype=torch.float32)
        y = torch.tensor(y.values, dtype=torch.long)
        y = F.one_hot(y, num_classes=3).float()
        
        data = torch.cat([X, y], dim=1)
        return data, y

In [6]:
from random import shuffle

# # Assuming 'total_df' contains the stock data and 'segment_pairs' is a list of (stock_id, period_indices) pairs
max_window_size = 128

# Assuming 'df' and 'num_classes' are defined
train_indices, test_indices = segment_pairs[:int(0.8 * len(segment_pairs))], segment_pairs[int(0.8 * len(segment_pairs)):]

trainset = SequentialDataset(df=total_df, segment_pairs=train_indices, max_window_size=max_window_size)
testset = SequentialDataset(df=total_df, segment_pairs=test_indices, max_window_size=max_window_size)

print('Train indices: ', len(train_indices))
print('Test indices: ', len(test_indices))

print(trainset[0][0].shape)
print(len(trainset))

Train indices:  3926
Test indices:  982
torch.Size([71, 15])
6984787


In [7]:
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from nn.utils.init import set_random_seed
set_random_seed(0)

c:\Users\CCNets-team\anaconda3\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
data_config = DataConfig(dataset_name = 'stock_price', task_type='multi_class_classification', obs_shape=[15], label_size=3)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'gpt', encoder_network = 'none')
ml_params.training.num_epoch = 100
ml_params.training.batch_size = 64

first_data = trainset[0]
X, y = first_data

print(f"Input shape: {X.shape}")
print(f"Label shape: {y.shape}")

print(f"Total number of samples in trainset: {len(trainset)}")

Input shape: torch.Size([112, 15])
Label shape: torch.Size([112, 3])
Total number of samples in trainset: 6984787


In [9]:
from trainer_hub import TrainerHub

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)

In [10]:
trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/109137 [00:00<?, ?it/s]

[0/100][50/109137][Time 19.47]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0424	Gen: 0.3491	Rec: 0.3428	E: 0.0479	R: 0.0353	P: 0.6502
--------------------Test Metrics------------------------
accuracy: 0.9531
precision: 0.3177
recall: 0.3333
f1_score: 0.3253

[0/100][100/109137][Time 18.52]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0201	Gen: 0.2491	Rec: 0.2519	E: 0.0173	R: 0.0229	P: 0.4840
--------------------Test Metrics------------------------
accuracy: 0.9766
precision: 0.6588
recall: 0.6667
f1_score: 0.6627

[0/100][150/109137][Time 18.56]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0110	Gen: 0.2305	Rec: 0.2325	E: 0.0086	R: 0.0126	P: 0.4418
--------------------Test Metrics------------------------
accuracy: 0.9844
